# Homework 7: Baseball Mini-Project
Amanda Kuznecov (anr431)

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as st
from scipy.special import logit, expit
import statsmodels.formula.api as smf
import statsmodels.api as sm
from statsmodels.tools.eval_measures import mse
from sklearn.metrics import brier_score_loss
import statsmodels.api as sm
from tqdm import tqdm
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)

In [2]:
#read in data
df_sc = pd.read_parquet('sc19.parquet')
df_sc = df_sc.loc[df_sc.game_type == 'R'].copy()
df_sc = df_sc.drop(columns = 'des').sort_values(['game_date','game_pk','at_bat_number','pitch_number'])

#remove any rows where there is no outcome
df_sc = df_sc.loc[df_sc.events.notnull()]

#removed features from model because they aren't actual pitches
col_list = ['intent_walk','ejection','game_advisory','wild_pitch','pickoff_1b','pickoff_2b','pickoff_caught_stealing_2b',
           'pickoff_caught_stealing_home','pickoff_caught_stealing_3b','pickoff_3b','pickoff_error_2b']

df_sc = df_sc.loc[~df_sc.events.isin(col_list)]

#remove rows where 4 balls occur
df_sc = df_sc.loc[df_sc.balls !=4]

df_play = pd.read_csv('MLB_players19.csv')

## Question 1: Expanded Baseline Model

In [3]:
#group for pitching stats per game
group_game = df_sc.groupby(['player_name','pitcher','game_pk','game_year'])

#get number of balls and number of batters faced per game per pitcher
game_summary = group_game.agg(
    bb = pd.NamedAgg(column = 'events', aggfunc = lambda x: ((x == 'walk')).sum()),
    k = pd.NamedAgg(column = 'events', aggfunc = lambda x: ((x == 'strikeout')|(x == 'strikeout_double_play')).sum()),
    bf = pd.NamedAgg(column = 'at_bat_number', aggfunc = 'nunique')
    )
game_summary = game_summary.reset_index().sort_values(['player_name','pitcher','game_pk','game_year']) 

In [4]:
game_summary.head()

,player_name,pitcher,game_pk,game_year,bb,k,bf
0,"Aardsma, David",430911,320068,2012,1,1,5
1,"Aardsma, David",430911,347669,2013,0,1,3
2,"Aardsma, David",430911,347713,2013,0,2,5
3,"Aardsma, David",430911,347746,2013,0,2,4
4,"Aardsma, David",430911,347776,2013,0,1,3


In [5]:
#group for pitching stats per season
group_year = game_summary.groupby(['player_name','pitcher','game_year'])

#get number of balls and number of batters faced per season per pitcher
yr_summary = group_year.agg(
    bb = pd.NamedAgg(column = 'bb', aggfunc = 'sum'),
    k = pd.NamedAgg(column = 'k', aggfunc = 'sum'),
    bf = pd.NamedAgg(column = 'bf', aggfunc = 'sum')
    )
yr_summary = yr_summary.reset_index().sort_values(['player_name','pitcher','game_year']) 

In [6]:
#get previous batters faced
yr_summary['bf_prev'] = yr_summary.groupby(['player_name','pitcher'])['bf'].transform(lambda x: x.shift(1,fill_value = 0))

#add walks rate feature
yr_summary.loc[:,'bbrate'] = yr_summary.bb/yr_summary.bf

#get previous walks rate
yr_summary['bbrate_prev'] = yr_summary.groupby(['player_name','pitcher'])['bbrate'].transform(lambda x: x.shift(1,fill_value = 0))


#add strikeouts rate feature
yr_summary.loc[:,'krate'] = yr_summary.k/(yr_summary.bf-yr_summary.bb)

#get previous strikeouts rate
yr_summary['krate_prev'] = yr_summary.groupby(['player_name','pitcher'])['krate'].transform(lambda x: x.shift(1,fill_value = 0))

#check how many seasons ago pitcher played (ie. 1 means they played last season, 0 means they haven't played yet)
yr_summary['played_prev'] = yr_summary.groupby(['player_name','pitcher'])['game_year'].transform(lambda x: x-x.shift(1,fill_value = x.min()))

In [7]:
yr_summary.head()

,player_name,pitcher,game_year,bb,k,bf,bf_prev,bbrate,bbrate_prev,krate,krate_prev,played_prev
0,"Aardsma, David",430911,2012,1,1,5,0,0.200000,0.000000,0.250000,0.000000,0
1,"Aardsma, David",430911,2013,13,36,172,5,0.075581,0.200000,0.226415,0.250000,1
2,"Aardsma, David",430911,2015,11,35,125,172,0.088000,0.075581,0.307018,0.226415,2
3,"Abad, Fernando",472551,2012,18,38,207,0,0.086957,0.000000,0.201058,0.000000,0
4,"Abad, Fernando",472551,2013,10,32,166,207,0.060241,0.086957,0.205128,0.201058,1


### Part a.i: bb-rate

In [8]:
#filter for seasons other than 2012 and 2019, bf at least 200, and stats based on consecutive seasons only
data = yr_summary.loc[(yr_summary.game_year != 2012) & (yr_summary.game_year != 2019) & (yr_summary.bf >=200) & (yr_summary.bf_prev >=200)& (yr_summary.played_prev <= 1)]
data.head()

,player_name,pitcher,game_year,bb,k,bf,bf_prev,bbrate,bbrate_prev,krate,krate_prev,played_prev
6,"Abad, Fernando",472551,2015,16,45,204,214,0.078431,0.056075,0.239362,0.252475,1
37,"Adleman, Tim",534947,2017,50,108,531,286,0.094162,0.066434,0.224532,0.176030,1
51,"Albers, Matt",458006,2013,20,35,261,239,0.076628,0.079498,0.145228,0.200000,1
55,"Albers, Matt",458006,2017,17,63,233,235,0.072961,0.076596,0.291667,0.138249,1
61,"Alburquerque, Al",456379,2014,20,63,235,215,0.085106,0.134884,0.293023,0.376344,1


In [9]:
#bbrate model
mod_bb = smf.ols('bbrate~bbrate_prev',data = data).fit()
mod_bb.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 bbrate   R-squared:                       0.294
Model:                            OLS   Adj. R-squared:                  0.293
Method:                 Least Squares   F-statistic:                     593.4
Date:                Wed, 14 Apr 2021   Prob (F-statistic):          7.21e-110
Time:                        08:56:21   Log-Likelihood:                 3619.4
No. Observations:                1428   AIC:                            -7235.
Df Residuals:                    1426   BIC:                            -7224.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       0.0332      0.002     19.342      0.000       0.030       0.037
bbrate_prev     0.5537      0.023     24.359      0.000       0.509       0.598
==============================================================================
Omnibus:                       72.252   Durbin-Watson:                   2.120
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               98.115
Skew:                           0.468   Prob(JB):                     4.95e-22
Kurtosis:                       3.879   Cond. No.                         45.0
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Part a.ii

In [10]:
#filter for 2019 season, bf at least 200, and stats based on consecutive seasons only
data_19 = yr_summary.loc[(yr_summary.game_year == 2019) & (yr_summary.bf >=200) & (yr_summary.bf_prev >=200)& (yr_summary.played_prev <= 1)]

In [11]:
bbrate_preds = mod_bb.predict(data_19.bbrate_prev)
bb_mse = mse(data_19.bbrate, bbrate_preds)
print(f'Out-of-sample MSE: {bb_mse}')

Out-of-sample MSE: 0.00046098357531102147


### Part b.i: k-rate

In [12]:
#krate model
mod_k = smf.ols('krate~krate_prev',data = data).fit()
mod_k.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  krate   R-squared:                       0.550
Model:                            OLS   Adj. R-squared:                  0.550
Method:                 Least Squares   F-statistic:                     1745.
Date:                Wed, 14 Apr 2021   Prob (F-statistic):          9.16e-250
Time:                        08:57:20   Log-Likelihood:                 2455.8
No. Observations:                1428   AIC:                            -4908.
Df Residuals:                    1426   BIC:                            -4897.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0602      0.004     13.854      0.000       0.052       0.069
krate_prev     0.7447      0.018     41.776      0.000       0.710       0.780
==============================================================================
Omnibus:                      103.116   Durbin-Watson:                   2.214
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              140.450
Skew:                           0.608   Prob(JB):                     3.17e-31
Kurtosis:                       3.938   Cond. No.                         16.4
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Part b.ii

In [13]:
krate_preds = mod_k.predict(data_19.krate_prev)
k_mse = mse(data_19.krate, krate_preds)
print(f'Out-of-sample MSE: {k_mse}')

Out-of-sample MSE: 0.001996196008974319


## Question 2: Called Strike Model

In [14]:
df_cs = df_sc.copy()

#get columns of interest
df_cs = df_cs[['game_year','description','pitch_type','batter','release_pos_x','release_pos_z','zone','stand','p_throws','balls',
              'strikes','pfx_x','pfx_z','plate_x','plate_z','vx0','vy0','sz_top','sz_bot','vz0','ax','ay','az',
              'outs_when_up','on_1b','on_2b','on_3b','release_speed','release_spin_rate','release_extension']]

#create indicator variable for when called strike
df_cs.loc[:,'called_strike'] = (df_cs.description == 'called_strike')*1.0

#fill missing values with -999
df_cs = df_cs.fillna(-999)

In [15]:
#create indicator variable for if pitcher is throwing to the outside of the batter
df_cs['outside_pitch'] = (df_cs['stand'] == df_cs['p_throws'])*1.0

#remove stand and pitcher columns
df_cs = df_cs.drop(columns = ['stand','p_throws'])

#create interaction variable between whether pitcher was pitching outside and plate_x
df_cs['outside_pitch_platex'] = df_cs.outside_pitch*df_cs.plate_x

In [16]:
#treat categorical columns
cat_cols = ['pitch_type','zone']

for col in cat_cols:
    for val in df_cs[col].unique():
        if val != -999:
            df_cs[col+'_'+str(val)] = (df_cs[col] == val)*1.0
        else:
            #indicator variable for if column is missing value
            df_cs[col+'_mv'] = (df_cs[col] == -999)*1.0
            
#drop original categorical columns
df_cs = df_cs.drop(columns = cat_cols)

In [17]:
#treat on base indicator columns
on_b = ['on_1b','on_2b','on_3b']

#create indicator variable for whether there is a player on base or not
for b in on_b:
    df_cs[b+'_ind'] = np.where(df_cs[b] == -999,0.0,1.0)

#drop original on base columns
df_cs = df_cs.drop(columns = on_b)

In [18]:
#these columns are always missing together: release_pos, pfx, plate, v, sz, a
#create column indicating if these are missing
df_cs['num_mv'] = np.where(df_cs.release_pos_x == -999,1.0,0.0)

#create columns indicating if remaining release variables are missing
df_cs['release_speed_mv'] = np.where(df_cs.release_speed == -999,1.0,0.0)
df_cs['release_spin_rate_mv'] = np.where(df_cs.release_spin_rate == -999,1.0,0.0)
df_cs['release_extension_mv'] = np.where(df_cs.release_extension == -999,1.0,0.0)

In [19]:
#treat missing numerical data

num_cols = ['release_pos_x','release_pos_z','pfx_x','pfx_z','plate_x','plate_z','vx0','vy0',
           'vz0','sz_top','sz_bot','ax','ay','az','release_speed','release_extension']

#replace nan values in numerical columns with mean of column
for col in num_cols:
    df_cs[col] = df_cs[col].replace([-999],df_cs.loc[df_cs[col] !=-999][col].mean())

#release spin rate col is an int so treat separately
df_cs['release_spin_rate'] = df_cs['release_spin_rate'].replace([-999],int(df_cs.loc[df_cs['release_spin_rate'] !=-999]['release_spin_rate'].mean()))
df_cs['release_spin_rate'] = df_cs['release_spin_rate'].astype(float)


In [20]:
#apply linear penalties for pitches outside strike zone
def HighMiss(z, top, thresh):
    return np.maximum(0, z-(top-thresh))

def LowMiss(z, bot, thresh):
    return np.maximum(bot+thresh-z, 0)

def LeftMiss(x, thresh):
    return np.maximum(0, -x-thresh)

def RightMiss(x, thresh):
    return np.maximum(0, x-thresh)

In [22]:
#feature for distance from pitch to centre of strike zone
df_cs['dist_mid'] = np.sqrt(df_cs.plate_x**2+((df_cs.sz_top + df_cs.sz_bot)/2 - df_cs.plate_z)**2)

In [23]:
#if fastball, never called strike so don't include as feature
df_cs.groupby(['pitch_type_FA']).called_strike.value_counts()/df_cs.groupby(['pitch_type_FA']).called_strike.count()

pitch_type_FA  called_strike
0.0            0.0              0.950176
               1.0              0.049824
1.0            0.0              1.000000
Name: called_strike, dtype: float64

In [24]:
#if pitch out, never called strike so don't include as feature
df_cs.groupby(['pitch_type_PO']).called_strike.value_counts()/df_cs.groupby(['pitch_type_PO']).called_strike.count()

pitch_type_PO  called_strike
0.0            0.0              0.950174
               1.0              0.049826
1.0            0.0              1.000000
Name: called_strike, dtype: float64

In [25]:
#drop fastball/pitch out columns
df_cs = df_cs.drop(columns = ['pitch_type_FA','pitch_type_PO'])

In [26]:
#incorporate indcator variables for ball and strike counts
for b in df_cs['balls'].unique():
    for s in df_cs['strikes'].unique():
        df_cs['b'+str(b)+'s'+str(s)] = ((df_cs['balls'] == b) &(df_cs['strikes'] == s))*1.0

In [27]:
#incorporate indicator variable for number of outs when up
for val in df_cs['outs_when_up'].unique():
    df_cs['outs_when_up' +str(val)] = (df_cs['outs_when_up'] == val)*1.0

In [28]:
#restrict only to pitches where batter did not swing; ie. ball, blocked ball, or called strike
data_cs = df_cs.loc[(df_cs.description == 'ball')|(df_cs.description == 'blocked_ball')|(df_cs.description == 'called_strike')]

#split into train/test
train = data_cs.loc[data_cs.game_year < 2018].copy()
test = data_cs.loc[data_cs.game_year == 2018].copy()

y_train = train.called_strike
y_test = test.called_strike

### Model 1

In [29]:
mod_cs1 = smf.logit('called_strike ~ HighMiss(plate_z,sz_top,0.25) + LowMiss(plate_z,sz_bot,0.25) + LeftMiss(plate_x,0.75)+RightMiss(plate_x,0.75)', data = train).fit()


Optimization terminated successfully.
         Current function value: 0.285160
         Iterations 9


In [30]:
#all features have predictive power
mod_cs1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:          called_strike   No. Observations:               137679
Model:                          Logit   Df Residuals:                   137674
Method:                           MLE   Df Model:                            4
Date:                Wed, 14 Apr 2021   Pseudo R-squ.:                  0.5744
Time:                        08:59:06   Log-Likelihood:                -39261.
converged:                       True   LL-Null:                       -92237.
Covariance Type:            nonrobust   LLR p-value:                     0.000
===================================================================================================
                                      coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
Intercept                           2.5377      0.016    161.575      0.000       2.507       2.568
HighMiss(plate_z, sz_top, 0.25)    -9.2302      0.084   -109.743      0.000      -9.395      -9.065
LowMiss(plate_z, sz_bot, 0.25)     -8.3928      0.063   -133.948      0.000      -8.516      -8.270
LeftMiss(plate_x, 0.75)            -7.7622      0.060   -129.230      0.000      -7.880      -7.645
RightMiss(plate_x, 0.75)          -10.5477      0.092   -115.152      0.000     -10.727     -10.368
===================================================================================================
"""

In [31]:
y_preds = mod_cs1.predict(test.drop(columns = 'called_strike'))
brier_score_loss(y_test, y_preds)

0.07777594460030494

### Model 2

In [32]:
mod_cs2 = smf.logit('called_strike ~ HighMiss(plate_z,sz_top,0.25) + LowMiss(plate_z,sz_bot,0.25) + LeftMiss(plate_x,0.75)+RightMiss(plate_x,0.75)+dist_mid+plate_x+outside_pitch_platex', data = train).fit()


Optimization terminated successfully.
         Current function value: 0.279425
         Iterations 9


In [33]:
#improved model performance with new features
mod_cs2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:          called_strike   No. Observations:               137679
Model:                          Logit   Df Residuals:                   137671
Method:                           MLE   Df Model:                            7
Date:                Wed, 14 Apr 2021   Pseudo R-squ.:                  0.5829
Time:                        08:59:12   Log-Likelihood:                -38471.
converged:                       True   LL-Null:                       -92237.
Covariance Type:            nonrobust   LLR p-value:                     0.000
===================================================================================================
                                      coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
Intercept                           3.5914      0.061     58.859      0.000       3.472       3.711
HighMiss(plate_z, sz_top, 0.25)    -8.1890      0.115    -71.466      0.000      -8.414      -7.964
LowMiss(plate_z, sz_bot, 0.25)     -7.2312      0.101    -71.732      0.000      -7.429      -7.034
LeftMiss(plate_x, 0.75)            -7.5939      0.127    -59.951      0.000      -7.842      -7.346
RightMiss(plate_x, 0.75)           -7.8423      0.146    -53.807      0.000      -8.128      -7.557
dist_mid                           -1.2784      0.076    -16.745      0.000      -1.428      -1.129
plate_x                            -0.3915      0.022    -18.049      0.000      -0.434      -0.349
outside_pitch_platex                0.0038      0.000     36.884      0.000       0.004       0.004
===================================================================================================
"""

In [34]:
y_preds = mod_cs2.predict(test.drop(columns = 'called_strike'))
brier_score_loss(y_test, y_preds)

0.07771236163288979

### Model 3

In [35]:
mod_cs3 = smf.logit('called_strike ~ HighMiss(plate_z,sz_top,0.25) + LowMiss(plate_z,sz_bot,0.25) + LeftMiss(plate_x,0.75)+RightMiss(plate_x,0.75)+dist_mid+plate_x+outside_pitch_platex+b0s1+b0s2+b1s1+b1s2+b3s1+b3s0+b3s2+b2s1+b2s0+b2s2', data = train).fit()

Optimization terminated successfully.
         Current function value: 0.087362
         Iterations 13


In [36]:
#ball/strike counts significantly improve model but some lower counts are not helpful
mod_cs3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:          called_strike   No. Observations:               137679
Model:                          Logit   Df Residuals:                   137661
Method:                           MLE   Df Model:                           17
Date:                Wed, 14 Apr 2021   Pseudo R-squ.:                  0.8696
Time:                        09:04:51   Log-Likelihood:                -12028.
converged:                       True   LL-Null:                       -92237.
Covariance Type:            nonrobust   LLR p-value:                     0.000
===================================================================================================
                                      coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
Intercept                           2.3468      0.164     14.337      0.000       2.026       2.668
HighMiss(plate_z, sz_top, 0.25)    -8.1310      0.198    -41.142      0.000      -8.518      -7.744
LowMiss(plate_z, sz_bot, 0.25)     -6.9117      0.167    -41.314      0.000      -7.240      -6.584
LeftMiss(plate_x, 0.75)            -8.8525      0.231    -38.306      0.000      -9.305      -8.400
RightMiss(plate_x, 0.75)           -9.0510      0.273    -33.113      0.000      -9.587      -8.515
dist_mid                           -0.9849      0.125     -7.869      0.000      -1.230      -0.740
plate_x                            -0.5446      0.038    -14.262      0.000      -0.619      -0.470
outside_pitch_platex                0.0042      0.000     19.524      0.000       0.004       0.005
b0s1                               -0.7172      0.301     -2.382      0.017      -1.307      -0.127
b0s2                                6.2887      0.201     31.287      0.000       5.895       6.683
b1s1                               -0.3730      0.310     -1.201      0.230      -0.982       0.236
b1s2                                6.5326      0.184     35.536      0.000       6.172       6.893
b3s1                               -6.1097      0.301    -20.285      0.000      -6.700      -5.519
b3s0                               -6.9910      0.519    -13.469      0.000      -8.008      -5.974
b3s2                                0.2700      0.138      1.955      0.051      -0.001       0.541
b2s1                               -0.5614      0.515     -1.090      0.276      -1.571       0.448
b2s0                                0.0474      0.569      0.083      0.934      -1.067       1.162
b2s2                                6.8079      0.195     34.966      0.000       6.426       7.189
===================================================================================================

Possibly complete quasi-separation: A fraction 0.27 of observations can be
perfectly predicted. This might indicate that there is complete
quasi-separation. In this case some parameters will not be identified.
"""

In [37]:
y_preds = mod_cs3.predict(test.drop(columns = 'called_strike'))
brier_score_loss(y_test, y_preds)

0.025380950527243647

### Model 4

In [38]:
mod_cs4 = smf.logit('called_strike ~ HighMiss(plate_z,sz_top,0.25) + LowMiss(plate_z,sz_bot,0.25) + LeftMiss(plate_x,0.75)+RightMiss(plate_x,0.75)+dist_mid+plate_x+b0s2+b1s2+b3s1+b3s0+b2s2+num_mv', data = train).fit()


Optimization terminated successfully.
         Current function value: 0.083099
         Iterations 13


In [39]:
#achieved better performance without the low ball/strike count features
mod_cs4.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:          called_strike   No. Observations:               137679
Model:                          Logit   Df Residuals:                   137666
Method:                           MLE   Df Model:                           12
Date:                Wed, 14 Apr 2021   Pseudo R-squ.:                  0.8760
Time:                        09:05:53   Log-Likelihood:                -11441.
converged:                       True   LL-Null:                       -92237.
Covariance Type:            nonrobust   LLR p-value:                     0.000
===================================================================================================
                                      coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
Intercept                           4.7483      0.131     36.381      0.000       4.492       5.004
HighMiss(plate_z, sz_top, 0.25)    -6.1267      0.207    -29.555      0.000      -6.533      -5.720
LowMiss(plate_z, sz_bot, 0.25)     -4.8695      0.179    -27.258      0.000      -5.220      -4.519
LeftMiss(plate_x, 0.75)            -6.5763      0.243    -27.015      0.000      -7.053      -6.099
RightMiss(plate_x, 0.75)           -6.4500      0.285    -22.625      0.000      -7.009      -5.891
dist_mid                           -3.5067      0.158    -22.218      0.000      -3.816      -3.197
plate_x                            -0.5936      0.039    -15.067      0.000      -0.671      -0.516
b0s2                                6.0522      0.148     40.993      0.000       5.763       6.342
b1s2                                6.2997      0.122     51.538      0.000       6.060       6.539
b3s1                               -6.2575      0.270    -23.219      0.000      -6.786      -5.729
b3s0                               -7.2463      0.502    -14.448      0.000      -8.229      -6.263
b2s2                                6.5751      0.138     47.661      0.000       6.305       6.845
num_mv                             -5.6661      0.158    -35.888      0.000      -5.976      -5.357
===================================================================================================

Possibly complete quasi-separation: A fraction 0.32 of observations can be
perfectly predicted. This might indicate that there is complete
quasi-separation. In this case some parameters will not be identified.
"""

In [40]:
y_preds = mod_cs4.predict(test.drop(columns = 'called_strike'))
brier_score_loss(y_test, y_preds)

0.02426679502706984

## Question 3: Swinging Strike Model

In [41]:
df_ss = df_cs.copy()

#create binary column for swinging strike
df_ss.loc[:,'swinging_strike'] = ((df_ss.description == 'swinging_strike')
                                  |(df_ss.description == 'swinging_strike_blocked'))*1.0

In [42]:
#nearly all 3 ball, 1 strike counts will end in no swinging strike
df_ss.groupby(['b3s1']).swinging_strike.value_counts()/df_ss.groupby(['b3s1']).swinging_strike.count()


b3s1  swinging_strike
0.0   0.0                0.845029
      1.0                0.154971
1.0   0.0                0.999784
      1.0                0.000216
Name: swinging_strike, dtype: float64

In [43]:
#all 3 ball, 0 strike counts will always end in no swinging strike
df_ss.groupby(['b3s0']).swinging_strike.value_counts()/df_ss.groupby(['b3s0']).swinging_strike.count()

b3s0  swinging_strike
0.0   0.0                0.849567
      1.0                0.150433
1.0   0.0                1.000000
Name: swinging_strike, dtype: float64

In [44]:
df_ss = df_ss.drop(columns = ['b3s1','b3s0'])

In [45]:
#split into train/test
train = df_ss.loc[df_ss.game_year < 2018].copy()
test = df_ss.loc[df_ss.game_year == 2018].copy()

y_train = train.swinging_strike
X_train = train.drop(columns = 'swinging_strike')

y_test = test.swinging_strike
X_test = test.drop(columns = 'swinging_strike')

### Model 1

In [46]:
mod_ss1 = smf.logit('swinging_strike ~ HighMiss(plate_z,sz_top,0.25) + LowMiss(plate_z,sz_bot,0.25) + LeftMiss(plate_x,0.75)+RightMiss(plate_x,0.75)+dist_mid+plate_x+outside_pitch_platex+b0s1+b0s0+b0s2+b1s1+b1s0+b1s2+b3s2+b2s1+b2s0+b2s2+vx0+vy0+vz0', data = train).fit()


Optimization terminated successfully.
         Current function value: 0.267216
         Iterations 13


In [47]:
mod_ss1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:        swinging_strike   No. Observations:              1102054
Model:                          Logit   Df Residuals:                  1102033
Method:                           MLE   Df Model:                           20
Date:                Wed, 14 Apr 2021   Pseudo R-squ.:                  0.3516
Time:                        09:11:35   Log-Likelihood:            -2.9449e+05
converged:                       True   LL-Null:                   -4.5414e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
===================================================================================================
                                      coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
Intercept                         -10.8322      0.286    -37.831      0.000     -11.393     -10.271
HighMiss(plate_z, sz_top, 0.25)    -0.1407      0.025     -5.707      0.000      -0.189      -0.092
LowMiss(plate_z, sz_bot, 0.25)      0.6453      0.021     30.479      0.000       0.604       0.687
LeftMiss(plate_x, 0.75)            -1.1174      0.028    -39.266      0.000      -1.173      -1.062
RightMiss(plate_x, 0.75)           -0.8231      0.029    -28.583      0.000      -0.880      -0.767
dist_mid                            1.3635      0.018     74.588      0.000       1.328       1.399
plate_x                             0.0869      0.007     12.147      0.000       0.073       0.101
outside_pitch_platex               -0.0013   4.85e-05    -26.760      0.000      -0.001      -0.001
b0s1                                0.9218      0.340      2.709      0.007       0.255       1.589
b0s0                                0.7852      0.342      2.297      0.022       0.115       1.455
b0s2                                8.2851      0.278     29.810      0.000       7.740       8.830
b1s1                                1.2055      0.331      3.642      0.000       0.557       1.854
b1s0                                0.5236      0.410      1.276      0.202      -0.280       1.328
b1s2                                8.2305      0.278     29.617      0.000       7.686       8.775
b3s2                                7.1352      0.278     25.674      0.000       6.590       7.680
b2s1                                0.8036      0.410      1.960      0.050   -8.12e-05       1.607
b2s0                                0.5783      0.572      1.011      0.312      -0.543       1.699
b2s2                                8.1438      0.278     29.304      0.000       7.599       8.689
vx0                                -0.0018      0.001     -3.054      0.002      -0.003      -0.001
vy0                                -0.0050      0.001     -9.023      0.000      -0.006      -0.004
vz0                                 0.0461      0.002     26.865      0.000       0.043       0.049
===================================================================================================
"""

In [48]:
y_preds = mod_ss1.predict(test.drop(columns = 'swinging_strike'))
brier_score_loss(y_test, y_preds)

0.09362594035662831

### Model 2

In [49]:
mod_ss2 = smf.logit('swinging_strike ~ HighMiss(plate_z,sz_top,0.25) + LowMiss(plate_z,sz_bot,0.25) + LeftMiss(plate_x,0.75)+RightMiss(plate_x,0.75)+dist_mid+plate_x+outside_pitch_platex+b0s2+b1s2+b3s2+b2s2+vx0+vy0+vz0', data = train).fit()


Optimization terminated successfully.
         Current function value: 0.267223
         Iterations 12


In [50]:
mod_ss2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:        swinging_strike   No. Observations:              1102054
Model:                          Logit   Df Residuals:                  1102039
Method:                           MLE   Df Model:                           14
Date:                Wed, 14 Apr 2021   Pseudo R-squ.:                  0.3515
Time:                        09:19:06   Log-Likelihood:            -2.9449e+05
converged:                       True   LL-Null:                   -4.5414e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
===================================================================================================
                                      coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
Intercept                         -10.0895      0.113    -88.897      0.000     -10.312      -9.867
HighMiss(plate_z, sz_top, 0.25)    -0.1411      0.025     -5.724      0.000      -0.189      -0.093
LowMiss(plate_z, sz_bot, 0.25)      0.6447      0.021     30.461      0.000       0.603       0.686
LeftMiss(plate_x, 0.75)            -1.1176      0.028    -39.286      0.000      -1.173      -1.062
RightMiss(plate_x, 0.75)           -0.8231      0.029    -28.588      0.000      -0.880      -0.767
dist_mid                            1.3634      0.018     74.592      0.000       1.328       1.399
plate_x                             0.0869      0.007     12.142      0.000       0.073       0.101
outside_pitch_platex               -0.0013   4.85e-05    -26.757      0.000      -0.001      -0.001
b0s2                                7.5441      0.091     82.699      0.000       7.365       7.723
b1s2                                7.4896      0.091     82.196      0.000       7.311       7.668
b3s2                                6.3943      0.091     70.072      0.000       6.215       6.573
b2s2                                7.4028      0.091     81.222      0.000       7.224       7.581
vx0                                -0.0018      0.001     -3.051      0.002      -0.003      -0.001
vy0                                -0.0050      0.001     -9.002      0.000      -0.006      -0.004
vz0                                 0.0461      0.002     26.870      0.000       0.043       0.049
===================================================================================================
"""

In [51]:
y_preds = mod_ss2.predict(test.drop(columns = 'swinging_strike'))
brier_score_loss(y_test, y_preds)

0.09362656950292164

### Model 3

In [52]:
mod_ss3 = smf.logit('swinging_strike ~ HighMiss(plate_z,sz_top,0.25) + LowMiss(plate_z,sz_bot,0.25) + LeftMiss(plate_x,0.75)+RightMiss(plate_x,0.75)+dist_mid+plate_x+outside_pitch_platex+b0s2+b1s2+b3s2+b2s2+vx0+vy0+vz0+release_speed+release_spin_rate+pitch_type_mv+pitch_type_FF+pitch_type_SL+pitch_type_CH+pitch_type_SI+pitch_type_CU+pitch_type_KC+pitch_type_FC+pitch_type_FC+pitch_type_FS+pitch_type_KN+pitch_type_SC+pitch_type_EP+pitch_type_FO', data = train).fit()


Optimization terminated successfully.
         Current function value: 0.263686
         Iterations 12


In [53]:
mod_ss3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:        swinging_strike   No. Observations:              1102054
Model:                          Logit   Df Residuals:                  1102024
Method:                           MLE   Df Model:                           29
Date:                Wed, 14 Apr 2021   Pseudo R-squ.:                  0.3601
Time:                        09:21:16   Log-Likelihood:            -2.9060e+05
converged:                       True   LL-Null:                   -4.5414e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
===================================================================================================
                                      coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
Intercept                         -15.3989      0.140   -109.869      0.000     -15.674     -15.124
HighMiss(plate_z, sz_top, 0.25)    -0.1427      0.025     -5.674      0.000      -0.192      -0.093
LowMiss(plate_z, sz_bot, 0.25)      0.5825      0.022     26.998      0.000       0.540       0.625
LeftMiss(plate_x, 0.75)            -1.1199      0.029    -39.058      0.000      -1.176      -1.064
RightMiss(plate_x, 0.75)           -0.8408      0.029    -28.986      0.000      -0.898      -0.784
dist_mid                            1.3678      0.018     74.393      0.000       1.332       1.404
plate_x                             0.0828      0.007     11.357      0.000       0.068       0.097
outside_pitch_platex               -0.0011   5.28e-05    -20.930      0.000      -0.001      -0.001
b0s2                                7.4337      0.091     81.833      0.000       7.256       7.612
b1s2                                7.3766      0.091     81.301      0.000       7.199       7.554
b3s2                                6.3296      0.091     69.657      0.000       6.152       6.508
b2s2                                7.2963      0.091     80.397      0.000       7.118       7.474
vx0                                -0.0073      0.001    -11.738      0.000      -0.009      -0.006
vy0                                -0.1592      0.016     -9.788      0.000      -0.191      -0.127
vz0                                 0.0541      0.002     28.599      0.000       0.050       0.058
release_speed                      -0.1742      0.024     -7.377      0.000      -0.221      -0.128
release_spin_rate                   0.0002   1.33e-05     11.392      0.000       0.000       0.000
pitch_type_mv                       1.3078      0.073     17.970      0.000       1.165       1.450
pitch_type_FF                       0.4788      0.014     34.037      0.000       0.451       0.506
pitch_type_SL                       1.1420      0.017     67.902      0.000       1.109       1.175
pitch_type_CH                       1.1421      0.018     62.252      0.000       1.106       1.178
pitch_type_SI                       0.0450      0.019      2.420      0.016       0.009       0.081
pitch_type_CU                       1.0506      0.022     47.320      0.000       1.007       1.094
pitch_type_KC                       1.0062      0.026     38.690      0.000       0.955       1.057
pitch_type_FC                       0.6982      0.019     35.970      0.000       0.660       0.736
pitch_type_FS                       1.2091      0.026     46.764      0.000       1.158       1.260
pitch_type_KN                       1.2883      0.049     26.278      0.000       1.192       1.384
pitch_type_SC                       0.6069      0.330      1.837      0.066      -0.041       1.254
pitch_type_EP                       1.3319      0.150      8.871      0.000       1.038       1.626
pitch_type_FO                       1.023

In [54]:
y_preds = mod_ss3.predict(test.drop(columns = 'swinging_strike'))
brier_score_loss(y_test, y_preds)

0.09221831525128585

### Model 4

In [67]:
mod_ss4 = smf.logit('swinging_strike ~ HighMiss(plate_z,sz_top,0.25) + LowMiss(plate_z,sz_bot,0.25) + LeftMiss(plate_x,0.75)+RightMiss(plate_x,0.75)+dist_mid+plate_x+b0s2+b1s2+b3s2+b2s2+vx0+vy0+vz0+release_speed+release_spin_rate+pitch_type_mv+pitch_type_FF+pitch_type_SL+pitch_type_CH+pitch_type_CU+pitch_type_KC+pitch_type_FC+pitch_type_FC+pitch_type_FS+pitch_type_KN+pitch_type_EP+pitch_type_FO+outs_when_up2+sz_bot+sz_top+num_mv', data = train).fit()


Optimization terminated successfully.
         Current function value: 0.262766
         Iterations 12


In [68]:
mod_ss4.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:        swinging_strike   No. Observations:              1102054
Model:                          Logit   Df Residuals:                  1102023
Method:                           MLE   Df Model:                           30
Date:                Wed, 14 Apr 2021   Pseudo R-squ.:                  0.3624
Time:                        09:30:45   Log-Likelihood:            -2.8958e+05
converged:                       True   LL-Null:                   -4.5414e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
===================================================================================================
                                      coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
Intercept                         -15.5346      0.149   -104.358      0.000     -15.826     -15.243
HighMiss(plate_z, sz_top, 0.25)    -0.1193      0.026     -4.563      0.000      -0.170      -0.068
LowMiss(plate_z, sz_bot, 0.25)      0.6513      0.022     29.513      0.000       0.608       0.695
LeftMiss(plate_x, 0.75)            -1.0965      0.029    -37.696      0.000      -1.153      -1.039
RightMiss(plate_x, 0.75)           -0.7994      0.030    -27.064      0.000      -0.857      -0.742
dist_mid                            1.3484      0.019     70.953      0.000       1.311       1.386
plate_x                             0.0723      0.007      9.876      0.000       0.058       0.087
b0s2                                7.4513      0.091     81.618      0.000       7.272       7.630
b1s2                                7.3943      0.091     81.091      0.000       7.216       7.573
b3s2                                6.3577      0.091     69.613      0.000       6.179       6.537
b2s2                                7.3146      0.091     80.198      0.000       7.136       7.493
vx0                                -0.0074      0.001    -11.897      0.000      -0.009      -0.006
vy0                                -0.2242      0.016    -14.080      0.000      -0.255      -0.193
vz0                                 0.0548      0.002     28.733      0.000       0.051       0.059
release_speed                      -0.2692      0.023    -11.639      0.000      -0.315      -0.224
release_spin_rate                   0.0001   1.33e-05     10.582      0.000       0.000       0.000
pitch_type_mv                       0.7248      0.074      9.841      0.000       0.580       0.869
pitch_type_FF                       0.4576      0.011     40.270      0.000       0.435       0.480
pitch_type_SL                       1.0956      0.014     76.456      0.000       1.068       1.124
pitch_type_CH                       1.0988      0.016     68.177      0.000       1.067       1.130
pitch_type_CU                       0.9945      0.020     49.149      0.000       0.955       1.034
pitch_type_KC                       0.9461      0.024     38.742      0.000       0.898       0.994
pitch_type_FC                       0.6631      0.017     37.986      0.000       0.629       0.697
pitch_type_FS                       1.1629      0.024     47.744      0.000       1.115       1.211
pitch_type_KN                       1.2449      0.048     25.803      0.000       1.150       1.339
pitch_type_EP                       1.2749      0.150      8.482      0.000       0.980       1.569
pitch_type_FO                       0.9602      0.126      7.639      0.000       0.714       1.207
outs_when_up2                       0.1007      0.007     14.685      0.000       0.087       0.114
sz_bot                             -1.4963      0.041    -36.358      0.000      -1.577      -1.416
sz_top                              0.731

In [69]:
y_preds = mod_ss4.predict(test.drop(columns = 'swinging_strike'))
brier_score_loss(y_test, y_preds)

0.09125743956692411

## Question 4: Improving the Baseline

In [281]:
#probability of pitch being called a strike
df_cs['strike_pred'] = mod_cs4.predict(df_cs)

#create feature for strikes above expectation
df_cs['extra_strike'] = df_cs.called_strike - df_cs.strike_pred

In [282]:
#net ‘extra strikes’ on pitches where batter had 2 strikes
df_cs['xtr1S'] = df_cs.extra_strike*(df_cs.b0s2 + df_cs.b1s2 + df_cs.b2s2 +df_cs.b3s2)

#net ‘extra strikes’ on pitches where batter had 3 balls
df_cs['xtr1B'] = df_cs.extra_strike*(df_cs.b3s0 + df_cs.b3s1 + df_cs.b3s2)

In [283]:
df_cs_small = df_cs[['dist_mid', 'extra_strike','b0s2','b1s2','b2s2','b3s2','b3s0','b3s1','xtr1S','xtr1B']]

#get smaller df of original with columns missing from previous analysis
df_sc_small = df_sc[['player_name','pitcher','game_pk','game_year','events','balls','strikes','at_bat_number']]

#join with previously engineered features
df_feats = df_sc_small.join(df_cs_small, how='left')

#sort df
df_feats = df_feats.sort_values(['player_name','pitcher','game_pk','at_bat_number']) 

In [284]:
#create 3 ball counts and 2 strike counts
df_feats['b3_count'] = df_feats.b3s0 + df_feats.b3s1 + df_feats.b3s2
df_feats['s2_count'] = df_feats.b0s2 + df_feats.b1s2 + df_feats.b2s2 + df_feats.b3s2

In [285]:
#group for game level stats
group_game = df_feats.groupby(['player_name','pitcher','game_pk','game_year'])

game_stats = group_game.agg(
    balls = pd.NamedAgg(column = 'balls', aggfunc = 'sum'),
    strikes = pd.NamedAgg(column = 'strikes', aggfunc = 'sum'),
    bb = pd.NamedAgg(column = 'events', aggfunc = lambda x: (x == 'walk').sum()),
    k = pd.NamedAgg(column = 'events', aggfunc = lambda x: ((x == 'strikeout')|(x == 'strikeout_double_play')).sum()),
    bf = pd.NamedAgg(column = 'at_bat_number', aggfunc = 'nunique'), 
    dist_mid = pd.NamedAgg(column = 'dist_mid', aggfunc = 'mean'),
    extra_strike = pd.NamedAgg(column = 'extra_strike', aggfunc = 'sum'),
    b3_count = pd.NamedAgg(column = 'b3_count', aggfunc = 'sum'),
    s2_count = pd.NamedAgg(column = 's2_count', aggfunc = 'sum'),
    xtr1S = pd.NamedAgg(column = 'xtr1S', aggfunc = 'sum'),
    xtr1B = pd.NamedAgg(column = 'xtr1B', aggfunc = 'sum')
    )
game_stats  = game_stats .reset_index().sort_values(['player_name','pitcher','game_pk','game_year']) 

In [286]:
game_stats.head()

,player_name,pitcher,game_pk,game_year,balls,strikes,bb,k,bf,dist_mid,extra_strike,b3_count,s2_count,xtr1S,xtr1B
0,"Aardsma, David",430911,320068,2012,9,8,1,1,5,1.298633,-2.125359,2.0,4.0,-1.959447,-0.002619
1,"Aardsma, David",430911,347669,2013,5,5,0,1,3,1.013666,-2.119672,1.0,2.0,-1.567935,-0.572021
2,"Aardsma, David",430911,347713,2013,3,8,0,2,5,0.778057,-3.919330,0.0,3.0,-1.965191,0.000000
3,"Aardsma, David",430911,347746,2013,4,7,0,2,4,0.476956,-2.978430,0.0,3.0,-1.993060,0.000000
4,"Aardsma, David",430911,347776,2013,1,6,0,1,3,0.940016,-2.963401,0.0,3.0,-2.963401,0.000000


In [287]:
#group for pitching stats per season
group_year = game_stats.groupby(['player_name','pitcher','game_year'])

year_stats = group_year.agg(
    balls = pd.NamedAgg(column = 'balls', aggfunc = 'sum'),
    strikes = pd.NamedAgg(column = 'strikes', aggfunc = 'sum'),
    bb = pd.NamedAgg(column = 'bb', aggfunc = 'sum'),
    k = pd.NamedAgg(column = 'k', aggfunc = 'sum'),
    bf = pd.NamedAgg(column = 'bf', aggfunc = 'sum'), 
    dist_mid = pd.NamedAgg(column = 'dist_mid', aggfunc = 'mean'),
    extra_strike = pd.NamedAgg(column = 'extra_strike', aggfunc = 'sum'),
    b3_count = pd.NamedAgg(column = 'b3_count', aggfunc = 'sum'),
    s2_count = pd.NamedAgg(column = 's2_count', aggfunc = 'sum'),
    xtr1S = pd.NamedAgg(column = 'xtr1S', aggfunc = 'sum'),
    xtr1B = pd.NamedAgg(column = 'xtr1B', aggfunc = 'sum')
    )
year_stats = year_stats.reset_index().sort_values(['player_name','pitcher','game_year']) 

In [288]:
#check how many seasons ago pitcher played (ie. 1 means they played last season, 0 means they haven't played yet)
year_stats['played_prev'] = year_stats.groupby(['player_name','pitcher'])['game_year'].transform(lambda x: x-x.shift(1,fill_value = x.min()))

#get previous season's batters faced
year_stats['bf_prev'] = year_stats.groupby(['player_name','pitcher'])['bf'].transform(lambda x: x.shift(1,fill_value = 0))

#get previous season's avg dist_mid
year_stats['dist_mid_prev'] = year_stats.groupby(['player_name','pitcher'])['dist_mid'].transform(lambda x: x.shift(1,fill_value = 0))

In [289]:
#create seasonal rates and prev values for all new features (with bf as denominator)
feats = ['bb','balls','strikes','extra_strike','b3_count','s2_count','xtr1S', 'xtr1B']

for feat in feats:
    year_stats[feat+'_rate'] = year_stats[feat]/year_stats.bf
    year_stats[feat+'_rate_prev'] = year_stats.groupby(['player_name','pitcher'])[feat+'_rate'].transform(lambda x: x.shift(1,fill_value = 0))

In [291]:
#k has a different denominator so run separately

#add strikeouts rate feature
year_stats.loc[:,'k_rate'] = year_stats.k/(year_stats.bf-year_stats.bb)

#get previous strikeouts rate
year_stats['k_rate_prev'] = year_stats.groupby(['player_name','pitcher'])['k_rate'].transform(lambda x: x.shift(1,fill_value = 0))

In [292]:
#filter for seasons other than 2012 and 2019, bf at least 200, and stats based on consecutive seasons only
data_18 = year_stats.loc[(year_stats.game_year != 2012) & (year_stats.game_year != 2019) & (year_stats.bf >=200) & (year_stats.bf_prev >=200)& (year_stats.played_prev <= 1)]
data_19 = year_stats.loc[(year_stats.game_year == 2019) & (year_stats.bf >=200) & (year_stats.bf_prev >=200)& (year_stats.played_prev <= 1)]

### Part a: bb-rate

### Model 1

In [293]:
#features in expected direction, xtr1B not as significant as expected
mod_bb1 = smf.ols('bb_rate~bb_rate_prev+xtr1B_rate_prev+balls_rate_prev',data = data_18).fit()
mod_bb1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                bb_rate   R-squared:                       0.323
Model:                            OLS   Adj. R-squared:                  0.321
Method:                 Least Squares   F-statistic:                     226.1
Date:                Wed, 14 Apr 2021   Prob (F-statistic):          6.22e-120
Time:                        12:35:19   Log-Likelihood:                 3649.1
No. Observations:                1428   AIC:                            -7290.
Df Residuals:                    1424   BIC:                            -7269.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -0.0188      0.007     -2.666      0.008      -0.033      -0.005
bb_rate_prev        0.3634      0.034     10.779      0.000       0.297       0.430
xtr1B_rate_prev     0.0774      0.046      1.693      0.091      -0.012       0.167
balls_rate_prev     0.0542      0.008      7.010      0.000       0.039       0.069
==============================================================================
Omnibus:                       74.044   Durbin-Watson:                   2.106
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               97.307
Skew:                           0.489   Prob(JB):                     7.41e-22
Kurtosis:                       3.825   Cond. No.                         160.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [294]:
bbrate_preds = mod_bb1.predict(data_19)
bb_mse = mse(data_19.bb_rate, bbrate_preds)
print(f'Out-of-sample MSE: {bb_mse}')

Out-of-sample MSE: 0.00045094156679394826


### Model 2

In [295]:
#dist_mid borderline significant
mod_bb2 = smf.ols('bb_rate~bb_rate_prev+xtr1B_rate_prev+balls_rate_prev+dist_mid_prev',data = data_18).fit()
mod_bb2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                bb_rate   R-squared:                       0.323
Model:                            OLS   Adj. R-squared:                  0.321
Method:                 Least Squares   F-statistic:                     169.9
Date:                Wed, 14 Apr 2021   Prob (F-statistic):          5.70e-119
Time:                        12:35:22   Log-Likelihood:                 3649.7
No. Observations:                1428   AIC:                            -7289.
Df Residuals:                    1423   BIC:                            -7263.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -0.0265      0.010     -2.635      0.008      -0.046      -0.007
bb_rate_prev        0.3596      0.034     10.606      0.000       0.293       0.426
xtr1B_rate_prev     0.0651      0.047      1.383      0.167      -0.027       0.158
balls_rate_prev     0.0521      0.008      6.538      0.000       0.036       0.068
dist_mid_prev       0.0110      0.010      1.077      0.282      -0.009       0.031
==============================================================================
Omnibus:                       71.665   Durbin-Watson:                   2.104
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               92.970
Skew:                           0.482   Prob(JB):                     6.48e-21
Kurtosis:                       3.795   Cond. No.                         186.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [296]:
bbrate_preds = mod_bb2.predict(data_19)
bb_mse = mse(data_19.bb_rate, bbrate_preds)
print(f'Out-of-sample MSE: {bb_mse}')

Out-of-sample MSE: 0.0004502286026483403


### Model 3

In [297]:
#ball counts of 3 seem to improve model performance
mod_bb3 = smf.ols('bb_rate~bb_rate_prev+xtr1B_rate_prev+balls_rate_prev+b3_count_rate_prev',data = data_18).fit()
mod_bb3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                bb_rate   R-squared:                       0.327
Model:                            OLS   Adj. R-squared:                  0.325
Method:                 Least Squares   F-statistic:                     172.7
Date:                Wed, 14 Apr 2021   Prob (F-statistic):          1.22e-120
Time:                        12:35:25   Log-Likelihood:                 3653.5
No. Observations:                1428   AIC:                            -7297.
Df Residuals:                    1423   BIC:                            -7271.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept             -0.0063      0.008     -0.767      0.443      -0.022       0.010
bb_rate_prev           0.2647      0.047      5.612      0.000       0.172       0.357
xtr1B_rate_prev        0.1587      0.053      2.988      0.003       0.055       0.263
balls_rate_prev        0.0338      0.010      3.285      0.001       0.014       0.054
b3_count_rate_prev     0.1368      0.046      2.982      0.003       0.047       0.227
==============================================================================
Omnibus:                       67.832   Durbin-Watson:                   2.112
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               87.382
Skew:                           0.466   Prob(JB):                     1.06e-19
Kurtosis:                       3.774   Cond. No.                         240.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [298]:
bbrate_preds = mod_bb3.predict(data_19)
bb_mse = mse(data_19.bb_rate, bbrate_preds)
print(f'Out-of-sample MSE: {bb_mse}')

Out-of-sample MSE: 0.00044874993281658555


### Part b: k-rate

### Model 1

In [299]:
#xS rate in correct sign direction but is not statistically significant, same with strikes rate
mod_k1 = smf.ols('k_rate~k_rate_prev+xtr1S_rate_prev+strikes_rate_prev',data = data_18).fit()
mod_k1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 k_rate   R-squared:                       0.551
Model:                            OLS   Adj. R-squared:                  0.550
Method:                 Least Squares   F-statistic:                     583.1
Date:                Wed, 14 Apr 2021   Prob (F-statistic):          3.64e-247
Time:                        12:35:29   Log-Likelihood:                 2457.3
No. Observations:                1428   AIC:                            -4907.
Df Residuals:                    1424   BIC:                            -4885.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             0.0298      0.025      1.184      0.237      -0.020       0.079
k_rate_prev           0.7093      0.032     22.507      0.000       0.647       0.771
xtr1S_rate_prev      -0.0213      0.043     -0.501      0.616      -0.105       0.062
strikes_rate_prev     0.0238      0.031      0.777      0.437      -0.036       0.084
==============================================================================
Omnibus:                      103.636   Durbin-Watson:                   2.203
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              140.874
Skew:                           0.612   Prob(JB):                     2.57e-31
Kurtosis:                       3.934   Cond. No.                         82.8
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [301]:
krate_preds = mod_k1.predict(data_19)
k_mse = mse(data_19.k_rate, krate_preds)
print(f'Out-of-sample MSE: {k_mse}')

Out-of-sample MSE: 0.0019732748658064622


### Model 2

In [302]:
#2 strike counts help model slightly
mod_k2 = smf.ols('k_rate~k_rate_prev+s2_count_rate_prev',data = data_18).fit()
mod_k2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 k_rate   R-squared:                       0.552
Model:                            OLS   Adj. R-squared:                  0.551
Method:                 Least Squares   F-statistic:                     878.0
Date:                Wed, 14 Apr 2021   Prob (F-statistic):          3.32e-249
Time:                        12:35:37   Log-Likelihood:                 2458.5
No. Observations:                1428   AIC:                            -4911.
Df Residuals:                    1425   BIC:                            -4895.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept              0.0319      0.013      2.452      0.014       0.006       0.057
k_rate_prev            0.6788      0.034     20.201      0.000       0.613       0.745
s2_count_rate_prev     0.0845      0.037      2.314      0.021       0.013       0.156
==============================================================================
Omnibus:                      103.506   Durbin-Watson:                   2.199
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              140.881
Skew:                           0.610   Prob(JB):                     2.56e-31
Kurtosis:                       3.937   Cond. No.                         49.2
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [303]:
krate_preds = mod_k2.predict(data_19)
k_mse = mse(data_19.k_rate, krate_preds)
print(f'Out-of-sample MSE: {k_mse}')

Out-of-sample MSE: 0.001967649552504296
